In [20]:
import datetime
import json
import os
import sys
import time

import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt

import seaborn as sns
sns.set()

import warnings
warnings.filterwarnings("ignore")

In [12]:
top_songs = pd.Series(np.load("data/top_songs.npy", allow_pickle=True))

In [13]:
top_songs

0         spotify:track:0UaMYEvWZi0ZqiDOoHU3YI
1         spotify:track:6I9VzXrHxO9rA9A5euc8Ak
2         spotify:track:0WqIKmW4BTrj3eJFmnCKMv
3         spotify:track:1AWQoqb9bSvzTjaLralEkT
4         spotify:track:1lzr43nnXAijIGYnCT8M8H
                          ...                 
348553    spotify:track:6pUIJ3ts5x9xZfsYnCSPix
348554    spotify:track:41Y3BwrR9Ay7XN4yyz6wTG
348555    spotify:track:5riOeRTlLR0qZS9CW9n7KS
348556    spotify:track:6YbPCsu6ggRFe9OVJlSry9
348557    spotify:track:7qLhC1Ib6rfiAJlcssbPDU
Length: 348558, dtype: object

In [14]:
track2idx_lookup = {v: i for v, i in  zip(top_songs.values, top_songs.index)}
track2idx_lookup

{'spotify:track:0UaMYEvWZi0ZqiDOoHU3YI': 0,
 'spotify:track:6I9VzXrHxO9rA9A5euc8Ak': 1,
 'spotify:track:0WqIKmW4BTrj3eJFmnCKMv': 2,
 'spotify:track:1AWQoqb9bSvzTjaLralEkT': 3,
 'spotify:track:1lzr43nnXAijIGYnCT8M8H': 4,
 'spotify:track:0XUfyU2QviPAs6bxSpXYG4': 5,
 'spotify:track:68vgtRHr7iZHpzGpon6Jlo': 6,
 'spotify:track:3BxWKCI06eQ5Od8TY2JBeA': 7,
 'spotify:track:7H6ev70Weq6DdpZyyTmUXk': 8,
 'spotify:track:2PpruBYCo4H7WOBJ7Q2EwM': 9,
 'spotify:track:2gam98EZKrF9XuOkU13ApN': 10,
 'spotify:track:4Y45aqo9QMa57rDsAJv40A': 11,
 'spotify:track:1HwpWwa6bnqqRhK8agG4RS': 12,
 'spotify:track:20ORwCJusz4KS2PbTPVNKo': 13,
 'spotify:track:7k6IzwMGpxnRghE7YosnXT': 14,
 'spotify:track:1Bv0Yl01xBDZD4OQP93fyl': 15,
 'spotify:track:4omisSlTk6Dsq2iQD7MA07': 16,
 'spotify:track:7xYnUQigPoIDAMPVK79NEq': 17,
 'spotify:track:6d8A5sAx9TfdeseDvfWNHd': 18,
 'spotify:track:4pmc2AxSEq6g7hPVlJCPyP': 19,
 'spotify:track:215JYyyUnrJ98NK3KEwu6d': 20,
 'spotify:track:0uqPG793dkDDN7sCUJJIVC': 21,
 'spotify:track:19Js

In [15]:
idx2track_lookup = {i: v for v, i in  zip(top_songs.values, top_songs.index)}
idx2track_lookup

{0: 'spotify:track:0UaMYEvWZi0ZqiDOoHU3YI',
 1: 'spotify:track:6I9VzXrHxO9rA9A5euc8Ak',
 2: 'spotify:track:0WqIKmW4BTrj3eJFmnCKMv',
 3: 'spotify:track:1AWQoqb9bSvzTjaLralEkT',
 4: 'spotify:track:1lzr43nnXAijIGYnCT8M8H',
 5: 'spotify:track:0XUfyU2QviPAs6bxSpXYG4',
 6: 'spotify:track:68vgtRHr7iZHpzGpon6Jlo',
 7: 'spotify:track:3BxWKCI06eQ5Od8TY2JBeA',
 8: 'spotify:track:7H6ev70Weq6DdpZyyTmUXk',
 9: 'spotify:track:2PpruBYCo4H7WOBJ7Q2EwM',
 10: 'spotify:track:2gam98EZKrF9XuOkU13ApN',
 11: 'spotify:track:4Y45aqo9QMa57rDsAJv40A',
 12: 'spotify:track:1HwpWwa6bnqqRhK8agG4RS',
 13: 'spotify:track:20ORwCJusz4KS2PbTPVNKo',
 14: 'spotify:track:7k6IzwMGpxnRghE7YosnXT',
 15: 'spotify:track:1Bv0Yl01xBDZD4OQP93fyl',
 16: 'spotify:track:4omisSlTk6Dsq2iQD7MA07',
 17: 'spotify:track:7xYnUQigPoIDAMPVK79NEq',
 18: 'spotify:track:6d8A5sAx9TfdeseDvfWNHd',
 19: 'spotify:track:4pmc2AxSEq6g7hPVlJCPyP',
 20: 'spotify:track:215JYyyUnrJ98NK3KEwu6d',
 21: 'spotify:track:0uqPG793dkDDN7sCUJJIVC',
 22: 'spotify:track:

In [ ]:
overlaps = np.zeros((len(top_songs), len(top_songs)))

slices = [f"data/Songs/{f}" for f in os.listdir('data/Songs') if f.endswith(".csv")]
total_slices = 1000

start_time = time.time()

for i, slc in enumerate(slices[:total_slices + 1]): 
    slice_df = pd.read_csv(slc)
    for playlist_df in [group for _, group in slice_df.groupby("pid")]:
        tracks = playlist_df.track_uri
        for track1 in tracks:
            idx1 = track2idx_lookup.get(track1, None)
            if idx1 != None:
                for track2 in tracks: 
                    idx2 = track2idx_lookup.get(track2, None)
                    if idx2 != None and idx1 != idx2: 
                        overlaps[idx1][idx2] += 1
                overlaps[idx1][idx1] += 1
    current_time = time.time()
    elapsed_time = current_time - start_time
    time_left = total_slices * elapsed_time / float(i+1) - elapsed_time
    sys.stdout.write(
        f"\rParsed slice {i+1}/{total_slices}. Time remaining: {str(datetime.timedelta(seconds=time_left))[:-7]}")
    sys.stdout.flush()

Parsed slice 51/1000. Time remaining: 5:52:40

In [11]:
distances = np.zeros((len(top_songs), len(top_songs)))
for i, row in enumerate(overlaps):
    track_count = SongCounts[idx2track_lookup[i]]
    distances[i] = 1 - overlaps[i]/track_count

In [12]:
distances

array([[0.        , 0.66666667, 0.61904762, ..., 1.        , 1.        ,
        1.        ],
       [0.85106383, 0.        , 0.57446809, ..., 1.        , 1.        ,
        1.        ],
       [0.90243902, 0.75609756, 0.        , ..., 1.        , 1.        ,
        1.        ],
       ...,
       [1.        , 1.        , 1.        , ..., 0.        , 1.        ,
        1.        ],
       [1.        , 1.        , 1.        , ..., 1.        , 0.        ,
        1.        ],
       [1.        , 1.        , 1.        , ..., 1.        , 1.        ,
        0.        ]])